In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import dateutil
import datetime
import re
import matplotlib.pyplot as plt

import load_data

import pecos

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#test

In [ ]:
b = 'Capacity test Raw data- 2017-03-25_7 PM.csv'
c = 'Data_Extract_20170807-20170807_cap_test_daily_id38778c9c-115.csv'
raw_data = load_data.load_das_data('./data/', b)
#raw_data.columns.tolist()

In [ ]:
raw_data.shape

In [ ]:
# col_types = raw_data.apply(load_data.series_type, args=(load_data.type_defs,)).tolist()
# sub_types = raw_data.apply(load_data.series_type, args=(load_data.sub_type_defs,), bounds_check=False).tolist()
# irr_types = raw_data.apply(load_data.series_type, args=(load_data.irr_sensors_defs,), bounds_check=False).tolist()

# col_indices = ['-'.join([typ, sub_typ, irr_typ]) for typ, sub_typ, irr_typ in zip(col_types, sub_types, irr_types)]

# raw_data_clean_headers = raw_data.copy()
# raw_data_clean_headers.columns = col_indices

# eq_cnts = load_data.equip_counts(raw_data_clean_headers)

In [ ]:
trans = load_data.trans_dict(raw_data)

In [ ]:
trans_keys = list(trans.keys())
trans_keys.sort()
trans_keys

In [ ]:
pm = pecos.monitoring.PerformanceMonitoring()
system_name = 'test'
pm.add_dataframe(raw_data, system_name)
pm.add_translation_dictionary(trans, system_name)

In [ ]:
pm.check_timestamp(60)

In [ ]:
pm.check_missing()
pm.check_range([-5,1500], 'irr-ghi-pyran')
pm.check_range([-5,1500], 'irr-poa-pyran')
pm.check_range([-5,1500], 'irr-poa-ref_cell')
pm.check_range([0,10], 'op_state-inv-')
pm.check_range([-1,1], 'pf-inv')
pm.check_range([-10,40], 'temp-amb-')
pm.check_range([-10,127], 'temp-mod-')
pm.check_range([0,18], 'wind--')

In [ ]:
for key in trans_keys:
    pm.check_increment([0.00001, None], key)

In [ ]:
mask = pm.get_test_results_mask()
QCI = pecos.metrics.qci(mask)

In [ ]:
# Define output file names and directories
results_directory = 'Results'
if not os.path.exists(results_directory):
    os.makedirs(results_directory)
graphics_file_rootname = os.path.join(results_directory, 'test_results')
custom_graphics_file = os.path.abspath(os.path.join(results_directory, 'custom.png'))
metrics_file = os.path.join(results_directory, system_name + '_metrics.csv')
test_results_file = os.path.join(results_directory, system_name + '_test_results.csv')
report_file =  os.path.join(results_directory, system_name + '.html')

# Generate graphics
test_results_graphics = pecos.graphics.plot_test_results(graphics_file_rootname, pm)
plt.figure(figsize = (7.0,3.5))
ax = plt.gca()
raw_data.plot(ax=ax, ylim=[-1.5,1.5])
plt.savefig(custom_graphics_file, format='png', dpi=500)

# Write metrics, test results, and report files
pecos.io.write_metrics(metrics_file, QCI)
pecos.io.write_test_results(test_results_file, pm.test_results)
pecos.io.write_monitoring_report(report_file, pm, test_results_graphics, [custom_graphics_file], QCI)

In [ ]:
trans['irr-ghi-pyran']

In [ ]:
pm.df[pm.trans['irr-ghi-pyran']].head()

In [ ]:
len(raw_data_clean_headers.columns.tolist())

In [ ]:
len(eq_cnts)

In [ ]:
n_col_names = list(zip(col_indices, eq_cnts))
raw_data_clean_headers.columns = pd.MultiIndex.from_tuples(n_col_names, names=['label','equip_cnt'])

In [ ]:
raw_data_clean_headers.head()

In [ ]:
raw_data_clean_headers_f1 = load_data.apply_filter(raw_data_clean_headers, 'label', skip_strs=['inv'])

In [ ]:
raw_data_clean_headers_f1.head()